# Task 1
You are asked to build a three-layer feed-forward neural network to solve the monitoring problem of injection molding machine. Your implementation must be in Pytorch and executable in Google Colab environments. The proportion of training and testing samples is 70:30 where your model must deliver the smallest testing error possible. In that case, you need to select the number of nodes of hidden
layers, the number of epochs, the learning rates, the mini-batch size, etc. that lead to the smallest testing error. In this assignment, you have to use the SGD optimizer as exemplified in the lab materials under the mini-batch update fashion. The evaluation metric here is the classification error. No feature selection is allowed here.

## Import OQC.mat file

In [ ]:
from google.colab import files
uploaded = files.upload()

from scipy.io import loadmat
oqc = loadmat('OQC.mat')

Saving OQC.mat to OQC.mat


In [ ]:
oqc.keys()

dict_keys(['__header__', '__version__', '__globals__', 'data'])

In [ ]:
type(oqc['data']), oqc['data'].shape

(numpy.ndarray, (2952, 49))

In [ ]:
type(oqc['data'][0][0]), oqc['data'][0][0].shape

(numpy.float64, ())

## Load Data into Python DataFrame


In [ ]:
import pandas as pd
columns = ['1','2', '3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','43','44','45','46','47','48','Defect']
df = pd.DataFrame(oqc['data'], columns=columns)
df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,Defect
0,0.390525,0.147798,0.156398,0.002560,0.555556,0.777778,0.555556,0.714286,0.50,0.500000,0.0,0.0,0.350680,0.995993,0.787486,1.0,0.4375,0.375,1.0,0.333333,0.25,0.078025,0.306230,1.000000,0.5,0.066666,0.693770,0.614525,0.385475,0.921937,1.0,0.033465,0.049981,0.233803,1.0,0.980563,0.467565,0.616027,0.070283,0.719864,0.037255,0.329627,1.0,0.000038,0.125,0.078970,0.079625,0.922469,2.0
1,0.390525,0.147798,0.156398,0.002560,0.444444,0.666667,0.555556,0.714286,0.25,0.500000,0.0,0.0,0.350680,0.995993,0.787486,1.0,0.5000,0.500,0.5,0.666667,0.25,0.078025,0.306230,1.000000,0.5,0.066666,0.693770,0.614525,0.385475,0.921937,1.0,0.068898,0.049981,0.233803,1.0,0.918348,0.451597,0.616027,0.096282,0.719864,0.072549,0.328711,1.0,0.000038,0.125,0.053677,0.053677,0.921482,2.0
2,0.508494,0.147798,0.156398,0.002560,0.444444,0.666667,0.555556,0.714286,0.25,0.500000,0.0,0.0,0.331065,0.995993,0.787486,1.0,0.5000,0.500,0.5,0.666667,0.25,0.079506,0.305174,1.000000,0.5,0.066666,0.693770,0.614525,0.385475,0.920455,1.0,0.996063,0.049981,0.233803,1.0,0.623919,0.493513,0.635777,0.054857,0.842105,0.994118,0.326533,1.0,0.000038,0.125,0.056112,0.056112,0.920494,2.0
3,0.577954,0.147798,0.156398,0.002560,0.444444,0.666667,0.444444,0.571429,0.50,0.500000,0.0,0.0,0.327862,0.995993,0.787486,1.0,0.5000,0.375,0.0,0.666667,0.25,0.079012,0.306230,1.000000,0.5,0.066666,0.693770,0.614525,0.385475,0.920949,1.0,0.996063,0.049981,0.233803,1.0,0.670883,0.480539,0.637676,0.051998,0.877759,0.994118,0.326533,1.0,0.000038,0.125,0.056206,0.057424,0.920494,2.0
4,0.649490,0.147798,0.156398,0.002560,0.555556,0.666667,0.444444,0.571429,0.50,0.500000,0.0,0.0,0.331065,0.995993,0.787486,1.0,0.4375,0.375,0.0,0.666667,0.25,0.080000,0.306230,1.000000,0.5,0.066666,0.693770,0.614525,0.385475,0.919961,1.0,0.996063,0.049981,0.237900,1.0,0.649028,0.463074,0.634637,0.051911,0.874363,0.996078,0.323210,1.0,0.000038,0.125,0.057705,0.056112,0.920000,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,0.188184,0.009829,0.772512,0.000113,0.777778,0.555556,0.777778,0.857143,0.50,0.785714,1.0,1.0,0.350680,0.994391,0.977913,0.0,0.4375,0.500,0.5,0.666667,0.25,0.066667,0.305174,0.333333,0.5,0.166667,0.694826,0.616387,0.383613,0.933301,1.0,0.049213,0.049981,0.237388,1.0,0.983818,0.759980,0.616027,0.187971,0.767402,0.052941,0.272437,0.0,0.000038,0.125,0.203185,0.269508,0.933333,0.0
2948,0.471121,0.009829,0.772512,0.000113,0.666667,0.555556,0.666667,0.714286,0.25,0.785714,1.0,1.0,0.335468,0.994391,0.977913,0.0,0.4375,0.625,0.5,1.000000,0.25,0.068148,0.307286,0.333333,0.5,0.166667,0.692714,0.616387,0.383613,0.931818,1.0,0.996063,0.049981,0.230986,1.0,0.624291,0.472056,0.630839,0.055204,0.838710,0.998039,0.272437,0.0,0.000038,0.125,0.059672,0.059672,0.931852,0.0
2949,0.540015,0.009829,0.772512,0.000113,0.666667,0.555556,0.666667,0.714286,0.25,0.785714,1.0,1.0,0.335068,0.994391,0.977913,0.0,0.4375,0.625,0.5,1.000000,0.25,0.068642,0.305174,0.333333,0.5,0.166667,0.693770,0.616387,0.383613,0.931324,1.0,0.996063,0.049981,0.236364,1.0,0.670511,0.463074,0.630080,0.051911,0.853990,0.998039,0.271634,0.0,0.000038,0.125,0.056112,0.056112,0.931358,0.0
2950,0.606644,0.009829,0.772512,0.000113,0.555556,0.555556,0.444444,0.571429,0.25,0.857143,1.0,1.0,0.329463,0.994391,0.977913,0.0,0.5000,0.625,0.5,0.666667,0.25,0.069136,0.307286,0.333333,0.5,0.166667,0.692714,0.612663,0.387337,0.930830,1.0,0.996063,0.049981,0.233035,1.0,0.649772,0.466068,0.636156,0.052171,0.848896,0.998039,0.270832,0.0,0.000038,0.125,0.056581,0.056581,0.930864,0.0


In [ ]:
# Check distribution of Defect types
df["Defect"].value_counts()

1.0    1074
0.0    1008
2.0     870
Name: Defect, dtype: int64

## Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
import time

In [ ]:
X = df.iloc[:, 0:48]
Y = df.loc[:, "Defect"]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from random import randint
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
# Convert dataframe into Tensors
X_train, X_test = map(torch.tensor, (X_train.values, X_test.values))
Y_train, Y_test = map(torch.tensor, (Y_train.values, Y_test.values))

# Make the tensors dtype as float for X values
X_train = X_train.float()
X_test = X_test.float()

Y_train = Y_train.long()
Y_test = Y_test.long()

In [ ]:
print(X_train.size())
print(X_test.size())
print(Y_train.unique(return_counts=True))

torch.Size([2066, 48])
torch.Size([886, 48])
(tensor([0, 1, 2]), tensor([709, 747, 610]))


## Set hyper-parameters

In [ ]:
# Hyper-parameters 
input_size = X_train.shape[1]   # 48
output_size = len(Y_train.unique())   # 3 outcomes: 0, 1, 2
hidden_size = 200  # no of nodes in hidden layers
num_epochs = 150
batch_size = 20
learning_rate = 0.01

## Feed Forward Neural Network

In [ ]:
# Make a fully connected neural network with 3 layers
class FeedForward(nn.Module):

    def __init__(self, input_size, output_size, hidden_size):
        super(FeedForward, self).__init__()
        self.inputs = input_size
        self.outputs = output_size
        self.hidden = hidden_size

        # Define the layers
        self.fc1 = nn.Linear(self.inputs, self.hidden, bias = False)
        self.fc2 = nn.Linear(self.hidden, self.hidden, bias = False)
        self.fc3 = nn.Linear(self.hidden, self.outputs, bias = False)
        
    def forward(self, x):
        output = self.fc1(x)
        output = F.relu(output)
        output = self.fc2(output)
        output = F.relu(output)
        output = self.fc3(output)
        return output

In [ ]:
# Create network object
model = FeedForward(input_size, output_size, hidden_size)
print(model)

FeedForward(
  (fc1): Linear(in_features=48, out_features=200, bias=False)
  (fc2): Linear(in_features=200, out_features=200, bias=False)
  (fc3): Linear(in_features=200, out_features=3, bias=False)
)


##Train the Model and Get Classification error on Test set

In [ ]:
def get_error(outputs, labels):

    batch_size=outputs.size(0)
    predicted_labels = outputs.argmax(dim = 1)
    indicator = (predicted_labels == labels)
    num_matches = indicator.sum()
    
    return 1 - num_matches.float()/batch_size 

In [ ]:
def eval_on_test_set():

    running_error = 0
    num_batches = 0

    for i in range(0, X_test.size()[0], batch_size):

        minibatch_data = X_test[i:i+batch_size]
        minibatch_label= Y_test[i:i+batch_size]

        inputs = minibatch_data.view(minibatch_data.size(),784)
        outputs = model(inputs) 

        error = get_error(outputs, minibatch_label)
        running_error += error.item()

        num_batches += 1

    total_error = running_error/num_batches
    print("Test error: {:.5f} %".format(total_error * 100))

In [ ]:
# Create a loss criteria
criterion = nn.CrossEntropyLoss()

# Create an optimizer (SGD optimizer) 
optimizer = torch.optim.SGD(params = model.parameters() , lr = learning_rate)

In [ ]:
start = time.time()

# Train the model
for epoch in range(num_epochs):

    # learning rate strategy : divide the learning rate by 1.5 every 10 epochs
    if epoch % 10 == 0 and epoch > 10: 
        learning_rate = learning_rate / 1.5
    
    # create a new optimizer at the beginning of each epoch: give the current learning rate.   
    optimizer = torch.optim.SGD(params = model.parameters() , lr = learning_rate)
        
    running_loss = 0
    running_error = 0
    num_batches = 0
    
    shuffled_indices = torch.randperm(X_train.size()[0])
 
    for count in range(0, X_train.size()[0], batch_size):
        
        # forward and backward pass
        optimizer.zero_grad()
        
        indices=shuffled_indices[count:count+batch_size]
        minibatch_data =  X_train[indices]
        minibatch_label = Y_train[indices]

        inputs = minibatch_data.view(minibatch_data.size(),48)
        inputs.requires_grad_()
        outputs = model(inputs) 

        loss = criterion(outputs, minibatch_label)
        loss.backward()
        optimizer.step()
        
        # compute some stats
        running_loss += loss.detach().item()
        error = get_error(outputs.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches += 1
    
    
    # once the epoch is finished we divide the "running quantities"
    # by the number of batches
    
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed_time = time.time() - start
    
    # every 10 epoch we display the stats 
    # and compute the error rate on the test set  
    
    if epoch % 10 == 0 : 
    
        print(' ')
        
        print("For Num. of Epochs: {:.0f}, Time: {:.5f}, Loss: {:.5f}, Learning Rate: {:.5f}, Error: {:.5f} %".format(epoch, elapsed_time, total_loss, learning_rate, total_error * 100))
        
        eval_on_test_set()

        

 
For Num. of Epochs: 0, Time: 0.08222, Loss: 1.09322, Learning Rate: 0.01000, Error: 62.13141 %
Test error: 54.74074 %
 
For Num. of Epochs: 10, Time: 0.89152, Loss: 0.54099, Learning Rate: 0.01000, Error: 11.84295 %
Test error: 11.96296 %
 
For Num. of Epochs: 20, Time: 1.70519, Loss: 0.22454, Learning Rate: 0.00667, Error: 8.31731 %
Test error: 8.25926 %
 
For Num. of Epochs: 30, Time: 2.50954, Loss: 0.16676, Learning Rate: 0.00444, Error: 6.16987 %
Test error: 5.55556 %
 
For Num. of Epochs: 40, Time: 3.31022, Loss: 0.13404, Learning Rate: 0.00296, Error: 4.51923 %
Test error: 4.11111 %
 
For Num. of Epochs: 50, Time: 4.12123, Loss: 0.11918, Learning Rate: 0.00198, Error: 3.47756 %
Test error: 3.11111 %
 
For Num. of Epochs: 60, Time: 4.94557, Loss: 0.10768, Learning Rate: 0.00132, Error: 2.83654 %
Test error: 2.44445 %
 
For Num. of Epochs: 70, Time: 5.77425, Loss: 0.10098, Learning Rate: 0.00088, Error: 2.40385 %
Test error: 2.00000 %
 
For Num. of Epochs: 80, Time: 6.59813, Loss

In [ ]:
# Print Classification Error on Test set
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 150,     Time: 12.22337,     mini batch size: 20,     learning rate: 0.0001,     num of hidden nodes: 200
Test error: 1.77778 %


## Summary: 
### Final hyper parameters used: 

```
  1.   Number of nodes of hidden layers = 200
  2.   Number of epochs = 150
  3.   Learning rates = 0.01 (Adaptive learning rate that divides by 1.5 every 10 epochs)
  4.   Mini-batch size = 20
```

### Classification Error on Test set: 
    Test accuracy: 1.77778  %
    Time: 12.22337 s


# Task 2: 
You are asked to study the effect of network structure: hidden nodes, hidden layers to the classification performance. That is, you try different network configurations and understand the patterns. Your experiments have to be well-documented in your Jupyter notebook file and your report. It has to cover different aspects of network configurations such as shallow network, wide network, deep network etc.

## Changing the number of hidden nodes (using 3 layers)

In [ ]:
# Hyper-parameters 
input_size = X_train.shape[1]   # 48
output_size = len(Y_train.unique())   # 3 outcomes: 0, 1, 2
hidden_size = 300  # no of nodes in hidden layers
num_epochs = 100
batch_size = 30
learning_rate = 0.001

# Make a fully connected neural network with 3 layers
class FeedForward(nn.Module):

    def __init__(self, input_size, output_size, hidden_size):
        super(FeedForward, self).__init__()
        self.inputs = input_size
        self.outputs = output_size
        self.hidden = hidden_size

        # Define the layers
        self.fc1 = nn.Linear(self.inputs, self.hidden, bias = False)
        self.fc2 = nn.Linear(self.hidden, self.hidden, bias = False)
        self.fc3 = nn.Linear(self.hidden, self.outputs, bias = False)
        
    def forward(self, x):
        output = self.fc1(x)
        output = F.relu(output)
        output = self.fc2(output)
        output = F.relu(output)
        output = self.fc3(output)
        return output

In [ ]:
model = FeedForward(input_size, output_size, hidden_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model.parameters() , lr = learning_rate)

start = time.time()

# Train the model
for epoch in range(num_epochs):

    running_loss = 0
    running_error = 0
    num_batches = 0
    
    shuffled_indices = torch.randperm(X_train.size()[0])
 
    for count in range(0, X_train.size()[0], batch_size):

        # forward and backward pass
        optimizer.zero_grad()
        
        indices=shuffled_indices[count:count+batch_size]
        minibatch_data =  X_train[indices]
        minibatch_label = Y_train[indices]

        inputs = minibatch_data.view(minibatch_data.size(),48)
        inputs.requires_grad_()
        outputs = model(inputs) 

        loss = criterion(outputs, minibatch_label)
        loss.backward()
        optimizer.step()
        
        # compute some stats
        running_loss += loss.detach().item()
        error = get_error(outputs.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches += 1
    
    # once the epoch is finished we divide the "running quantities" by the number of batches
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed_time = time.time() - start
    
    # every 10 epoch we display the stats and compute the error rate on the test set  
    if epoch % 10 == 0 : 

        print("For Num. of Epochs: {:.0f}, Time: {:.5f}, Loss: {:.5f}, Learning Rate: {:.5f}, Error: {:.5f} %".format(epoch, elapsed_time, total_loss, learning_rate, total_error * 100))
        eval_on_test_set()

For Num. of Epochs: 0, Time: 0.08204, Loss: 1.10103, Learning Rate: 0.00100, Error: 70.92902 %
Test error: 70.16667 %
For Num. of Epochs: 10, Time: 0.95110, Loss: 1.08510, Learning Rate: 0.00100, Error: 62.74991 %
Test error: 61.98611 %
For Num. of Epochs: 20, Time: 1.81506, Loss: 1.07195, Learning Rate: 0.00100, Error: 57.72204 %
Test error: 56.76389 %
For Num. of Epochs: 30, Time: 2.67028, Loss: 1.05834, Learning Rate: 0.00100, Error: 48.34262 %
Test error: 48.66667 %
For Num. of Epochs: 40, Time: 3.51786, Loss: 1.04181, Learning Rate: 0.00100, Error: 44.18803 %
Test error: 43.77778 %
For Num. of Epochs: 50, Time: 4.38073, Loss: 1.02070, Learning Rate: 0.00100, Error: 36.87105 %
Test error: 38.56944 %
For Num. of Epochs: 60, Time: 5.24633, Loss: 0.99285, Learning Rate: 0.00100, Error: 29.83278 %
Test error: 30.72222 %
For Num. of Epochs: 70, Time: 6.14356, Loss: 0.95584, Learning Rate: 0.00100, Error: 19.23077 %
Test error: 17.87500 %
For Num. of Epochs: 80, Time: 6.99840, Loss: 0.90

In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 4.60161,     mini batch size: 30,     learning rate: 0.0010,     num of hidden nodes: 100
Test error: 33.05556 %


In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 5.05955,     mini batch size: 30,     learning rate: 0.0010,     num of hidden nodes: 150
Test error: 15.37500 %


In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 5.73307,     mini batch size: 30,     learning rate: 0.0010,     num of hidden nodes: 200
Test error: 14.95833 %


In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 8.61859,     mini batch size: 30,     learning rate: 0.0010,     num of hidden nodes: 300
Test error: 12.66667 %


In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 11.00542,     mini batch size: 30,     learning rate: 0.0010,     num of hidden nodes: 400
Test error: 11.08333 %


## Results: 
### Constant hyper parameters used: 

  1.   Number of epochs = 100
  2.   Learning rates = 0.001 (Fixed)
  3.   Mini-batch size = 30


### Classification Error on Test set for Number of Hidden Nodes: 
- num of hidden nodes: 100, Time: 4.60161, Test error: 33.05556 %
- num of hidden nodes: 150, Time: 5.05955, Test error: 15.37500 %
- num of hidden nodes: 200, Time: 5.73307, Test error: 14.95833 %
- num of hidden nodes: 300, Time: 8.61859, Test error: 12.66667 %
- num of hidden nodes: 400, Time: 11.00542, Test error: 11.08333 %




## Changing the number of hidden layers (using 100 hidden nodes)

### 1 Layer

In [ ]:
# Hyper-parameters 
input_size = X_train.shape[1]   # 48
output_size = len(Y_train.unique())   # 3 outcomes: 0, 1, 2
hidden_size = 100  # no of nodes in hidden layers
num_epochs = 100
batch_size = 30
learning_rate = 0.001

# Fully connected neural network with 1 layer
class FeedForward(nn.Module):

    def __init__(self, input_size, output_size):
        super(FeedForward, self).__init__()
        self.inputs = input_size
        self.outputs = output_size

        # Define the layers
        self.fc1 = nn.Linear(self.inputs, self.outputs, bias = False)
        
    def forward(self, x):
        output = self.fc1(x)
        return output

In [ ]:
model = FeedForward(input_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model.parameters() , lr = learning_rate)

start = time.time()

# Train the model
for epoch in range(num_epochs):

    running_loss = 0
    running_error = 0
    num_batches = 0
    
    shuffled_indices = torch.randperm(X_train.size()[0])
 
    for count in range(0, X_train.size()[0], batch_size):

        # forward and backward pass
        optimizer.zero_grad()
        
        indices=shuffled_indices[count:count+batch_size]
        minibatch_data =  X_train[indices]
        minibatch_label = Y_train[indices]

        inputs = minibatch_data.view(minibatch_data.size(),48)
        inputs.requires_grad_()
        outputs = model(inputs) 

        loss = criterion(outputs, minibatch_label)
        loss.backward()
        optimizer.step()
        
        # compute some stats
        running_loss += loss.detach().item()
        error = get_error(outputs.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches += 1
    
    # once the epoch is finished we divide the "running quantities" by the number of batches
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed_time = time.time() - start
    
    # every 10 epoch we display the stats and compute the error rate on the test set  
    if epoch % 10 == 0 : 

        print("For Num. of Epochs: {:.0f}, Time: {:.5f}, Loss: {:.5f}, Learning Rate: {:.5f}, Error: {:.5f} %".format(epoch, elapsed_time, total_loss, learning_rate, total_error * 100))
        eval_on_test_set()

For Num. of Epochs: 0, Time: 0.03039, Loss: 1.12992, Learning Rate: 0.00100, Error: 65.67447 %
Test error: 66.38889 %
For Num. of Epochs: 10, Time: 0.32311, Loss: 1.05257, Learning Rate: 0.00100, Error: 50.69862 %
Test error: 49.68055 %
For Num. of Epochs: 20, Time: 0.61068, Loss: 1.02334, Learning Rate: 0.00100, Error: 39.58380 %
Test error: 38.59722 %
For Num. of Epochs: 30, Time: 0.90983, Loss: 0.99619, Learning Rate: 0.00100, Error: 33.77926 %
Test error: 31.70833 %
For Num. of Epochs: 40, Time: 1.19396, Loss: 0.97075, Learning Rate: 0.00100, Error: 29.57637 %
Test error: 28.27778 %
For Num. of Epochs: 50, Time: 1.46618, Loss: 0.94692, Learning Rate: 0.00100, Error: 26.44370 %
Test error: 24.72222 %
For Num. of Epochs: 60, Time: 1.74402, Loss: 0.92439, Learning Rate: 0.00100, Error: 25.22111 %
Test error: 23.38889 %
For Num. of Epochs: 70, Time: 2.02792, Loss: 0.90340, Learning Rate: 0.00100, Error: 23.95764 %
Test error: 21.62500 %
For Num. of Epochs: 80, Time: 2.30471, Loss: 0.88

In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 2.83738,     mini batch size: 30,     learning rate: 0.0010,     num of hidden nodes: 100
Test error: 19.63889 %


### 2 Layers

In [ ]:
# Hyper-parameters 
input_size = X_train.shape[1]   # 48
output_size = len(Y_train.unique())   # 3 outcomes: 0, 1, 2
hidden_size = 100  # no of nodes in hidden layers
num_epochs = 100
batch_size = 30
learning_rate = 0.001

# Make a fully connected neural network with 2 layers
class FeedForward(nn.Module):

    def __init__(self, input_size, output_size, hidden_size):
        super(FeedForward, self).__init__()
        self.inputs = input_size
        self.outputs = output_size
        self.hidden = hidden_size

        # Define the layers
        self.fc1 = nn.Linear(self.inputs, self.hidden, bias = False)
        self.fc2 = nn.Linear(self.hidden, self.hidden, bias = False)
        
    def forward(self, x):
        output = self.fc1(x)
        output = F.relu(output)
        output = self.fc2(output)
        return output

In [ ]:
model = FeedForward(input_size, output_size, hidden_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model.parameters() , lr = learning_rate)

start = time.time()

# Train the model
for epoch in range(num_epochs):

    running_loss = 0
    running_error = 0
    num_batches = 0
    
    shuffled_indices = torch.randperm(X_train.size()[0])
 
    for count in range(0, X_train.size()[0], batch_size):

        # forward and backward pass
        optimizer.zero_grad()
        
        indices=shuffled_indices[count:count+batch_size]
        minibatch_data =  X_train[indices]
        minibatch_label = Y_train[indices]

        inputs = minibatch_data.view(minibatch_data.size(),48)
        inputs.requires_grad_()
        outputs = model(inputs) 

        loss = criterion(outputs, minibatch_label)
        loss.backward()
        optimizer.step()
        
        # compute some stats
        running_loss += loss.detach().item()
        error = get_error(outputs.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches += 1
    
    # once the epoch is finished we divide the "running quantities" by the number of batches
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed_time = time.time() - start
    
    # every 10 epoch we display the stats and compute the error rate on the test set  
    if epoch % 10 == 0 : 

        print("For Num. of Epochs: {:.0f}, Time: {:.5f}, Loss: {:.5f}, Learning Rate: {:.5f}, Error: {:.5f} %".format(epoch, elapsed_time, total_loss, learning_rate, total_error * 100))
        eval_on_test_set()

For Num. of Epochs: 0, Time: 0.04402, Loss: 4.59894, Learning Rate: 0.00100, Error: 100.00000 %
Test error: 99.45833 %
For Num. of Epochs: 10, Time: 0.46247, Loss: 1.72215, Learning Rate: 0.00100, Error: 52.06986 %
Test error: 54.30555 %
For Num. of Epochs: 20, Time: 0.86610, Loss: 1.12875, Learning Rate: 0.00100, Error: 46.75585 %
Test error: 51.63889 %
For Num. of Epochs: 30, Time: 1.31397, Loss: 1.05840, Learning Rate: 0.00100, Error: 43.30360 %
Test error: 43.88889 %
For Num. of Epochs: 40, Time: 1.74355, Loss: 1.01801, Learning Rate: 0.00100, Error: 36.96024 %
Test error: 36.34722 %
For Num. of Epochs: 50, Time: 2.16157, Loss: 0.98182, Learning Rate: 0.00100, Error: 30.26013 %
Test error: 30.37500 %
For Num. of Epochs: 60, Time: 2.58490, Loss: 0.94540, Learning Rate: 0.00100, Error: 26.22817 %
Test error: 24.37500 %
For Num. of Epochs: 70, Time: 2.99616, Loss: 0.90687, Learning Rate: 0.00100, Error: 21.90635 %
Test error: 20.27778 %
For Num. of Epochs: 80, Time: 3.40633, Loss: 0.8

In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 4.19657,     mini batch size: 30,     learning rate: 0.0010,     num of hidden nodes: 100
Test error: 16.05556 %


### 3 Layers

In [ ]:
# Hyper-parameters 
input_size = X_train.shape[1]   # 48
output_size = len(Y_train.unique())   # 3 outcomes: 0, 1, 2
hidden_size = 100  # no of nodes in hidden layers
num_epochs = 100
batch_size = 30
learning_rate = 0.001

# Make a fully connected neural network with 3 layers
class FeedForward(nn.Module):

    def __init__(self, input_size, output_size, hidden_size):
        super(FeedForward, self).__init__()
        self.inputs = input_size
        self.outputs = output_size
        self.hidden = hidden_size

        # Define the layers
        self.fc1 = nn.Linear(self.inputs, self.hidden, bias = False)
        self.fc2 = nn.Linear(self.hidden, self.hidden, bias = False)
        self.fc3 = nn.Linear(self.hidden, self.outputs, bias = False)
        
    def forward(self, x):
        output = self.fc1(x)
        output = F.relu(output)
        output = self.fc2(output)
        output = F.relu(output)
        output = self.fc3(output)
        return output

In [ ]:
model = FeedForward(input_size, output_size, hidden_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model.parameters() , lr = learning_rate)

start = time.time()

# Train the model
for epoch in range(num_epochs):

    running_loss = 0
    running_error = 0
    num_batches = 0
    
    shuffled_indices = torch.randperm(X_train.size()[0])
 
    for count in range(0, X_train.size()[0], batch_size):

        # forward and backward pass
        optimizer.zero_grad()
        
        indices=shuffled_indices[count:count+batch_size]
        minibatch_data =  X_train[indices]
        minibatch_label = Y_train[indices]

        inputs = minibatch_data.view(minibatch_data.size(),48)
        inputs.requires_grad_()
        outputs = model(inputs) 

        loss = criterion(outputs, minibatch_label)
        loss.backward()
        optimizer.step()
        
        # compute some stats
        running_loss += loss.detach().item()
        error = get_error(outputs.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches += 1
    
    # once the epoch is finished we divide the "running quantities" by the number of batches
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed_time = time.time() - start
    
    # every 10 epoch we display the stats and compute the error rate on the test set  
    if epoch % 10 == 0 : 

        print("For Num. of Epochs: {:.0f}, Time: {:.5f}, Loss: {:.5f}, Learning Rate: {:.5f}, Error: {:.5f} %".format(epoch, elapsed_time, total_loss, learning_rate, total_error * 100))
        eval_on_test_set()

For Num. of Epochs: 0, Time: 0.04664, Loss: 1.09888, Learning Rate: 0.00100, Error: 63.85358 %
Test error: 63.08333 %
For Num. of Epochs: 10, Time: 0.52427, Loss: 1.08936, Learning Rate: 0.00100, Error: 63.87588 %
Test error: 63.08333 %
For Num. of Epochs: 20, Time: 0.97953, Loss: 1.08042, Learning Rate: 0.00100, Error: 63.77555 %
Test error: 63.08333 %
For Num. of Epochs: 30, Time: 1.44412, Loss: 1.07111, Learning Rate: 0.00100, Error: 57.56596 %
Test error: 55.54167 %
For Num. of Epochs: 40, Time: 1.91863, Loss: 1.06063, Learning Rate: 0.00100, Error: 45.61873 %
Test error: 43.25000 %
For Num. of Epochs: 50, Time: 2.38326, Loss: 1.04785, Learning Rate: 0.00100, Error: 40.39390 %
Test error: 38.13889 %
For Num. of Epochs: 60, Time: 2.83489, Loss: 1.03168, Learning Rate: 0.00100, Error: 36.37681 %
Test error: 35.90278 %
For Num. of Epochs: 70, Time: 3.31162, Loss: 1.01074, Learning Rate: 0.00100, Error: 32.65329 %
Test error: 33.47222 %
For Num. of Epochs: 80, Time: 3.76549, Loss: 0.98

In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 4.65410,     mini batch size: 30,     learning rate: 0.0010,     num of hidden nodes: 100
Test error: 17.38889 %


### 4 Layers

In [ ]:
# Hyper-parameters 
input_size = X_train.shape[1]   # 48
output_size = len(Y_train.unique())   # 3 outcomes: 0, 1, 2
hidden_size = 100  # no of nodes in hidden layers
num_epochs = 100
batch_size = 30
learning_rate = 0.001

# Fully connected neural network with 4 layers
class FeedForward(nn.Module):

    def __init__(self, input_size, output_size, hidden_size):
        super(FeedForward, self).__init__()
        self.inputs = input_size
        self.outputs = output_size
        self.hidden = hidden_size

        # Define the layers
        self.fc1 = nn.Linear(self.inputs, self.hidden, bias = False)
        self.fc2 = nn.Linear(self.hidden, self.hidden, bias = False)
        self.fc3 = nn.Linear(self.hidden, self.hidden, bias = False)
        self.fc4 = nn.Linear(self.hidden, self.outputs, bias = False)
        
    def forward(self, x):
        output = self.fc1(x)
        output = F.relu(output)
        output = self.fc2(output)
        output = F.relu(output)
        output = self.fc3(output)
        output = F.relu(output)
        output = self.fc4(output)
        return output

In [ ]:
model = FeedForward(input_size, output_size, hidden_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model.parameters() , lr = learning_rate)

start = time.time()

# Train the model
for epoch in range(num_epochs):

    running_loss = 0
    running_error = 0
    num_batches = 0
    
    shuffled_indices = torch.randperm(X_train.size()[0])
 
    for count in range(0, X_train.size()[0], batch_size):

        # forward and backward pass
        optimizer.zero_grad()
        
        indices=shuffled_indices[count:count+batch_size]
        minibatch_data =  X_train[indices]
        minibatch_label = Y_train[indices]

        inputs = minibatch_data.view(minibatch_data.size(),48)
        inputs.requires_grad_()
        outputs = model(inputs) 

        loss = criterion(outputs, minibatch_label)
        loss.backward()
        optimizer.step()
        
        # compute some stats
        running_loss += loss.detach().item()
        error = get_error(outputs.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches += 1
    
    # once the epoch is finished we divide the "running quantities" by the number of batches
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed_time = time.time() - start
    
    # every 10 epoch we display the stats and compute the error rate on the test set  
    if epoch % 10 == 0 : 

        print("For Num. of Epochs: {:.0f}, Time: {:.5f}, Loss: {:.5f}, Learning Rate: {:.5f}, Error: {:.5f} %".format(epoch, elapsed_time, total_loss, learning_rate, total_error * 100))
        eval_on_test_set()

For Num. of Epochs: 0, Time: 0.05949, Loss: 1.10062, Learning Rate: 0.00100, Error: 68.85544 %
Test error: 70.87500 %
For Num. of Epochs: 10, Time: 0.63963, Loss: 1.09777, Learning Rate: 0.00100, Error: 62.06615 %
Test error: 61.87500 %
For Num. of Epochs: 20, Time: 1.19648, Loss: 1.09543, Learning Rate: 0.00100, Error: 62.51951 %
Test error: 61.41667 %
For Num. of Epochs: 30, Time: 1.76419, Loss: 1.09345, Learning Rate: 0.00100, Error: 61.52731 %
Test error: 60.30555 %
For Num. of Epochs: 40, Time: 2.32070, Loss: 1.09151, Learning Rate: 0.00100, Error: 59.11185 %
Test error: 58.63889 %
For Num. of Epochs: 50, Time: 2.88235, Loss: 1.08952, Learning Rate: 0.00100, Error: 56.72241 %
Test error: 57.08333 %
For Num. of Epochs: 60, Time: 3.43101, Loss: 1.08741, Learning Rate: 0.00100, Error: 54.61167 %
Test error: 53.66667 %
For Num. of Epochs: 70, Time: 3.99634, Loss: 1.08512, Learning Rate: 0.00100, Error: 50.78038 %
Test error: 50.11111 %
For Num. of Epochs: 80, Time: 4.55501, Loss: 1.08

In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 5.61082,     mini batch size: 30,     learning rate: 0.0010,     num of hidden nodes: 100
Test error: 43.34722 %


### 5 Layers

In [ ]:
# Hyper-parameters 
input_size = X_train.shape[1]   # 48
output_size = len(Y_train.unique())   # 3 outcomes: 0, 1, 2
hidden_size = 100  # no of nodes in hidden layers
num_epochs = 100
batch_size = 30
learning_rate = 0.001

# Fully connected neural network with 5 layers
class FeedForward(nn.Module):

    def __init__(self, input_size, output_size, hidden_size):
        super(FeedForward, self).__init__()
        self.inputs = input_size
        self.outputs = output_size
        self.hidden = hidden_size

        # Define the layers
        self.fc1 = nn.Linear(self.inputs, self.hidden, bias = False)
        self.fc2 = nn.Linear(self.hidden, self.hidden, bias = False)
        self.fc3 = nn.Linear(self.hidden, self.hidden, bias = False)
        self.fc4 = nn.Linear(self.hidden, self.hidden, bias = False)
        self.fc5 = nn.Linear(self.hidden, self.outputs, bias = False)
        
    def forward(self, x):
        output = self.fc1(x)
        output = F.relu(output)
        output = self.fc2(output)
        output = F.relu(output)
        output = self.fc3(output)
        output = F.relu(output)
        output = self.fc4(output)
        output = F.relu(output)
        output = self.fc5(output)
        return output

In [ ]:
model = FeedForward(input_size, output_size, hidden_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model.parameters() , lr = learning_rate)

start = time.time()

# Train the model
for epoch in range(num_epochs):

    running_loss = 0
    running_error = 0
    num_batches = 0
    
    shuffled_indices = torch.randperm(X_train.size()[0])
 
    for count in range(0, X_train.size()[0], batch_size):

        # forward and backward pass
        optimizer.zero_grad()
        
        indices=shuffled_indices[count:count+batch_size]
        minibatch_data =  X_train[indices]
        minibatch_label = Y_train[indices]

        inputs = minibatch_data.view(minibatch_data.size(),48)
        inputs.requires_grad_()
        outputs = model(inputs) 

        loss = criterion(outputs, minibatch_label)
        loss.backward()
        optimizer.step()
        
        # compute some stats
        running_loss += loss.detach().item()
        error = get_error(outputs.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches += 1
    
    # once the epoch is finished we divide the "running quantities" by the number of batches
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed_time = time.time() - start
    
    # every 10 epoch we display the stats and compute the error rate on the test set  
    if epoch % 10 == 0 : 

        print("For Num. of Epochs: {:.0f}, Time: {:.5f}, Loss: {:.5f}, Learning Rate: {:.5f}, Error: {:.5f} %".format(epoch, elapsed_time, total_loss, learning_rate, total_error * 100))
        eval_on_test_set()

For Num. of Epochs: 0, Time: 0.07276, Loss: 1.09826, Learning Rate: 0.00100, Error: 64.11371 %
Test error: 64.16667 %
For Num. of Epochs: 10, Time: 0.70652, Loss: 1.09800, Learning Rate: 0.00100, Error: 63.45968 %
Test error: 63.50000 %
For Num. of Epochs: 20, Time: 1.32923, Loss: 1.09772, Learning Rate: 0.00100, Error: 63.03976 %
Test error: 62.38889 %
For Num. of Epochs: 30, Time: 1.94784, Loss: 1.09744, Learning Rate: 0.00100, Error: 61.89521 %
Test error: 61.94444 %
For Num. of Epochs: 40, Time: 2.60291, Loss: 1.09714, Learning Rate: 0.00100, Error: 60.40134 %
Test error: 60.70833 %
For Num. of Epochs: 50, Time: 3.24247, Loss: 1.09684, Learning Rate: 0.00100, Error: 58.70308 %
Test error: 59.16667 %
For Num. of Epochs: 60, Time: 3.88233, Loss: 1.09650, Learning Rate: 0.00100, Error: 57.48049 %
Test error: 58.94444 %
For Num. of Epochs: 70, Time: 4.49720, Loss: 1.09614, Learning Rate: 0.00100, Error: 56.47715 %
Test error: 57.06944 %
For Num. of Epochs: 80, Time: 5.11447, Loss: 1.09

In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 6.28459,     mini batch size: 30,     learning rate: 0.0010,     num of hidden nodes: 100
Test error: 56.63889 %


## Increasing the learning rate to allow nnet to train faster

### 1 Layer

In [ ]:
# Hyper-parameters 
input_size = X_train.shape[1]   # 48
output_size = len(Y_train.unique())   # 3 outcomes: 0, 1, 2
hidden_size = 100  # no of nodes in hidden layers
num_epochs = 100
batch_size = 30
learning_rate = 0.01

# Fully connected neural network with 1 layer
class FeedForward(nn.Module):

    def __init__(self, input_size, output_size):
        super(FeedForward, self).__init__()
        self.inputs = input_size
        self.outputs = output_size

        # Define the layers
        self.fc1 = nn.Linear(self.inputs, self.outputs, bias = False)
        
    def forward(self, x):
        output = self.fc1(x)
        return output

In [ ]:
model = FeedForward(input_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model.parameters() , lr = learning_rate)

start = time.time()

# Train the model
for epoch in range(num_epochs):

    running_loss = 0
    running_error = 0
    num_batches = 0
    
    shuffled_indices = torch.randperm(X_train.size()[0])
 
    for count in range(0, X_train.size()[0], batch_size):

        # forward and backward pass
        optimizer.zero_grad()
        
        indices=shuffled_indices[count:count+batch_size]
        minibatch_data =  X_train[indices]
        minibatch_label = Y_train[indices]

        inputs = minibatch_data.view(minibatch_data.size(),48)
        inputs.requires_grad_()
        outputs = model(inputs) 

        loss = criterion(outputs, minibatch_label)
        loss.backward()
        optimizer.step()
        
        # compute some stats
        running_loss += loss.detach().item()
        error = get_error(outputs.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches += 1
    
    # once the epoch is finished we divide the "running quantities" by the number of batches
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed_time = time.time() - start
    
    # every 10 epoch we display the stats and compute the error rate on the test set  
    if epoch % 10 == 0 : 

        print("For Num. of Epochs: {:.0f}, Time: {:.5f}, Loss: {:.5f}, Learning Rate: {:.5f}, Error: {:.5f} %".format(epoch, elapsed_time, total_loss, learning_rate, total_error * 100))
        eval_on_test_set()

For Num. of Epochs: 0, Time: 0.02961, Loss: 1.07539, Learning Rate: 0.01000, Error: 52.12560 %
Test error: 48.97222 %
For Num. of Epochs: 10, Time: 0.31884, Loss: 0.83949, Learning Rate: 0.01000, Error: 19.64326 %
Test error: 16.73611 %
For Num. of Epochs: 20, Time: 0.60527, Loss: 0.70700, Learning Rate: 0.01000, Error: 12.72761 %
Test error: 11.52778 %
For Num. of Epochs: 30, Time: 0.88669, Loss: 0.62224, Learning Rate: 0.01000, Error: 10.17466 %
Test error: 11.19445 %
For Num. of Epochs: 40, Time: 1.17868, Loss: 0.56231, Learning Rate: 0.01000, Error: 9.78818 %
Test error: 9.76389 %
For Num. of Epochs: 50, Time: 1.46798, Loss: 0.51789, Learning Rate: 0.01000, Error: 8.75883 %
Test error: 8.09722 %
For Num. of Epochs: 60, Time: 1.75185, Loss: 0.48325, Learning Rate: 0.01000, Error: 8.13824 %
Test error: 7.98611 %
For Num. of Epochs: 70, Time: 2.02709, Loss: 0.45482, Learning Rate: 0.01000, Error: 7.79264 %
Test error: 7.98611 %
For Num. of Epochs: 80, Time: 2.29686, Loss: 0.43142, Lea

In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 2.83572,     mini batch size: 30,     learning rate: 0.0100,     num of hidden nodes: 100
Test error: 7.09722 %


### 2 Layers

In [ ]:
# Hyper-parameters 
input_size = X_train.shape[1]   # 48
output_size = len(Y_train.unique())   # 3 outcomes: 0, 1, 2
hidden_size = 100  # no of nodes in hidden layers
num_epochs = 100
batch_size = 30
learning_rate = 0.01

# Make a fully connected neural network with 2 layers
class FeedForward(nn.Module):

    def __init__(self, input_size, output_size, hidden_size):
        super(FeedForward, self).__init__()
        self.inputs = input_size
        self.outputs = output_size
        self.hidden = hidden_size

        # Define the layers
        self.fc1 = nn.Linear(self.inputs, self.hidden, bias = False)
        self.fc2 = nn.Linear(self.hidden, self.hidden, bias = False)
        
    def forward(self, x):
        output = self.fc1(x)
        output = F.relu(output)
        output = self.fc2(output)
        return output

In [ ]:
model = FeedForward(input_size, output_size, hidden_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model.parameters() , lr = learning_rate)

start = time.time()

# Train the model
for epoch in range(num_epochs):

    running_loss = 0
    running_error = 0
    num_batches = 0
    
    shuffled_indices = torch.randperm(X_train.size()[0])
 
    for count in range(0, X_train.size()[0], batch_size):

        # forward and backward pass
        optimizer.zero_grad()
        
        indices=shuffled_indices[count:count+batch_size]
        minibatch_data =  X_train[indices]
        minibatch_label = Y_train[indices]

        inputs = minibatch_data.view(minibatch_data.size(),48)
        inputs.requires_grad_()
        outputs = model(inputs) 

        loss = criterion(outputs, minibatch_label)
        loss.backward()
        optimizer.step()
        
        # compute some stats
        running_loss += loss.detach().item()
        error = get_error(outputs.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches += 1
    
    # once the epoch is finished we divide the "running quantities" by the number of batches
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed_time = time.time() - start
    
    # every 10 epoch we display the stats and compute the error rate on the test set  
    if epoch % 10 == 0 : 

        print("For Num. of Epochs: {:.0f}, Time: {:.5f}, Loss: {:.5f}, Learning Rate: {:.5f}, Error: {:.5f} %".format(epoch, elapsed_time, total_loss, learning_rate, total_error * 100))
        eval_on_test_set()

For Num. of Epochs: 0, Time: 0.04440, Loss: 3.89981, Learning Rate: 0.01000, Error: 71.89892 %
Test error: 66.38889 %
For Num. of Epochs: 10, Time: 0.47811, Loss: 0.83930, Learning Rate: 0.01000, Error: 19.12672 %
Test error: 19.06945 %
For Num. of Epochs: 20, Time: 0.90281, Loss: 0.50557, Learning Rate: 0.01000, Error: 7.39874 %
Test error: 7.31945 %
For Num. of Epochs: 30, Time: 1.33247, Loss: 0.34587, Learning Rate: 0.01000, Error: 5.86771 %
Test error: 5.44445 %
For Num. of Epochs: 40, Time: 1.76552, Loss: 0.26461, Learning Rate: 0.01000, Error: 4.99814 %
Test error: 5.00000 %
For Num. of Epochs: 50, Time: 2.20631, Loss: 0.21472, Learning Rate: 0.01000, Error: 4.54850 %
Test error: 4.55556 %
For Num. of Epochs: 60, Time: 2.62222, Loss: 0.18136, Learning Rate: 0.01000, Error: 4.39614 %
Test error: 4.22222 %
For Num. of Epochs: 70, Time: 3.04918, Loss: 0.15671, Learning Rate: 0.01000, Error: 3.72724 %
Test error: 3.44445 %
For Num. of Epochs: 80, Time: 3.45664, Loss: 0.13836, Learnin

In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 4.24768,     mini batch size: 30,     learning rate: 0.0100,     num of hidden nodes: 100
Test error: 2.55556 %


### 3 Layers

In [ ]:
# Hyper-parameters 
input_size = X_train.shape[1]   # 48
output_size = len(Y_train.unique())   # 3 outcomes: 0, 1, 2
hidden_size = 100  # no of nodes in hidden layers
num_epochs = 100
batch_size = 30
learning_rate = 0.01

# Make a fully connected neural network with 3 layers
class FeedForward(nn.Module):

    def __init__(self, input_size, output_size, hidden_size):
        super(FeedForward, self).__init__()
        self.inputs = input_size
        self.outputs = output_size
        self.hidden = hidden_size

        # Define the layers
        self.fc1 = nn.Linear(self.inputs, self.hidden, bias = False)
        self.fc2 = nn.Linear(self.hidden, self.hidden, bias = False)
        self.fc3 = nn.Linear(self.hidden, self.outputs, bias = False)
        
    def forward(self, x):
        output = self.fc1(x)
        output = F.relu(output)
        output = self.fc2(output)
        output = F.relu(output)
        output = self.fc3(output)
        return output

In [ ]:
model = FeedForward(input_size, output_size, hidden_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model.parameters() , lr = learning_rate)

start = time.time()

# Train the model
for epoch in range(num_epochs):

    running_loss = 0
    running_error = 0
    num_batches = 0
    
    shuffled_indices = torch.randperm(X_train.size()[0])
 
    for count in range(0, X_train.size()[0], batch_size):

        # forward and backward pass
        optimizer.zero_grad()
        
        indices=shuffled_indices[count:count+batch_size]
        minibatch_data =  X_train[indices]
        minibatch_label = Y_train[indices]

        inputs = minibatch_data.view(minibatch_data.size(),48)
        inputs.requires_grad_()
        outputs = model(inputs) 

        loss = criterion(outputs, minibatch_label)
        loss.backward()
        optimizer.step()
        
        # compute some stats
        running_loss += loss.detach().item()
        error = get_error(outputs.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches += 1
    
    # once the epoch is finished we divide the "running quantities" by the number of batches
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed_time = time.time() - start
    
    # every 10 epoch we display the stats and compute the error rate on the test set  
    if epoch % 10 == 0 : 

        print("For Num. of Epochs: {:.0f}, Time: {:.5f}, Loss: {:.5f}, Learning Rate: {:.5f}, Error: {:.5f} %".format(epoch, elapsed_time, total_loss, learning_rate, total_error * 100))
        eval_on_test_set()

For Num. of Epochs: 0, Time: 0.04598, Loss: 1.09874, Learning Rate: 0.01000, Error: 58.92977 %
Test error: 50.76389 %
For Num. of Epochs: 10, Time: 0.50619, Loss: 0.95326, Learning Rate: 0.01000, Error: 24.65255 %
Test error: 21.69445 %
For Num. of Epochs: 20, Time: 0.94559, Loss: 0.42931, Learning Rate: 0.01000, Error: 12.45262 %
Test error: 13.65278 %
For Num. of Epochs: 30, Time: 1.41122, Loss: 0.25533, Learning Rate: 0.01000, Error: 10.25641 %
Test error: 8.87500 %
For Num. of Epochs: 40, Time: 1.87390, Loss: 0.19476, Learning Rate: 0.01000, Error: 7.65515 %
Test error: 7.98611 %
For Num. of Epochs: 50, Time: 2.34801, Loss: 0.15796, Learning Rate: 0.01000, Error: 6.23189 %
Test error: 6.22222 %
For Num. of Epochs: 60, Time: 2.80837, Loss: 0.12593, Learning Rate: 0.01000, Error: 4.54106 %
Test error: 4.44445 %
For Num. of Epochs: 70, Time: 3.25938, Loss: 0.10083, Learning Rate: 0.01000, Error: 2.89855 %
Test error: 2.77778 %
For Num. of Epochs: 80, Time: 3.72749, Loss: 0.07658, Lear

In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 4.61948,     mini batch size: 30,     learning rate: 0.0100,     num of hidden nodes: 100
Test error: 0.66667 %


### 4 Layers

In [ ]:
# Hyper-parameters 
input_size = X_train.shape[1]   # 48
output_size = len(Y_train.unique())   # 3 outcomes: 0, 1, 2
hidden_size = 100  # no of nodes in hidden layers
num_epochs = 100
batch_size = 30
learning_rate = 0.01

# Fully connected neural network with 4 layers
class FeedForward(nn.Module):

    def __init__(self, input_size, output_size, hidden_size):
        super(FeedForward, self).__init__()
        self.inputs = input_size
        self.outputs = output_size
        self.hidden = hidden_size

        # Define the layers
        self.fc1 = nn.Linear(self.inputs, self.hidden, bias = False)
        self.fc2 = nn.Linear(self.hidden, self.hidden, bias = False)
        self.fc3 = nn.Linear(self.hidden, self.hidden, bias = False)
        self.fc4 = nn.Linear(self.hidden, self.outputs, bias = False)
        
    def forward(self, x):
        output = self.fc1(x)
        output = F.relu(output)
        output = self.fc2(output)
        output = F.relu(output)
        output = self.fc3(output)
        output = F.relu(output)
        output = self.fc4(output)
        return output

In [ ]:
model = FeedForward(input_size, output_size, hidden_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model.parameters() , lr = learning_rate)

start = time.time()

# Train the model
for epoch in range(num_epochs):

    running_loss = 0
    running_error = 0
    num_batches = 0
    
    shuffled_indices = torch.randperm(X_train.size()[0])
 
    for count in range(0, X_train.size()[0], batch_size):

        # forward and backward pass
        optimizer.zero_grad()
        
        indices=shuffled_indices[count:count+batch_size]
        minibatch_data =  X_train[indices]
        minibatch_label = Y_train[indices]

        inputs = minibatch_data.view(minibatch_data.size(),48)
        inputs.requires_grad_()
        outputs = model(inputs) 

        loss = criterion(outputs, minibatch_label)
        loss.backward()
        optimizer.step()
        
        # compute some stats
        running_loss += loss.detach().item()
        error = get_error(outputs.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches += 1
    
    # once the epoch is finished we divide the "running quantities" by the number of batches
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed_time = time.time() - start
    
    # every 10 epoch we display the stats and compute the error rate on the test set  
    if epoch % 10 == 0 : 

        print("For Num. of Epochs: {:.0f}, Time: {:.5f}, Loss: {:.5f}, Learning Rate: {:.5f}, Error: {:.5f} %".format(epoch, elapsed_time, total_loss, learning_rate, total_error * 100))
        eval_on_test_set()

For Num. of Epochs: 0, Time: 0.05895, Loss: 1.09827, Learning Rate: 0.01000, Error: 65.54441 %
Test error: 65.94444 %
For Num. of Epochs: 10, Time: 0.63098, Loss: 1.08483, Learning Rate: 0.01000, Error: 65.28800 %
Test error: 65.94444 %
For Num. of Epochs: 20, Time: 1.16722, Loss: 1.00373, Learning Rate: 0.01000, Error: 58.19398 %
Test error: 56.75000 %
For Num. of Epochs: 30, Time: 1.71417, Loss: 0.43954, Learning Rate: 0.01000, Error: 11.92865 %
Test error: 12.63889 %
For Num. of Epochs: 40, Time: 2.25367, Loss: 0.21979, Learning Rate: 0.01000, Error: 8.93720 %
Test error: 8.41667 %
For Num. of Epochs: 50, Time: 2.79677, Loss: 0.16976, Learning Rate: 0.01000, Error: 7.36529 %
Test error: 8.22222 %
For Num. of Epochs: 60, Time: 3.34711, Loss: 0.12793, Learning Rate: 0.01000, Error: 5.65217 %
Test error: 4.98611 %
For Num. of Epochs: 70, Time: 3.88941, Loss: 0.10036, Learning Rate: 0.01000, Error: 3.81643 %
Test error: 3.22222 %
For Num. of Epochs: 80, Time: 4.42515, Loss: 0.06857, Lea

In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 5.48521,     mini batch size: 30,     learning rate: 0.0100,     num of hidden nodes: 100
Test error: 0.33333 %


### 5 Layers

In [ ]:
# Hyper-parameters 
input_size = X_train.shape[1]   # 48
output_size = len(Y_train.unique())   # 3 outcomes: 0, 1, 2
hidden_size = 100  # no of nodes in hidden layers
num_epochs = 100
batch_size = 30
learning_rate = 0.01

# Fully connected neural network with 5 layers
class FeedForward(nn.Module):

    def __init__(self, input_size, output_size, hidden_size):
        super(FeedForward, self).__init__()
        self.inputs = input_size
        self.outputs = output_size
        self.hidden = hidden_size

        # Define the layers
        self.fc1 = nn.Linear(self.inputs, self.hidden, bias = False)
        self.fc2 = nn.Linear(self.hidden, self.hidden, bias = False)
        self.fc3 = nn.Linear(self.hidden, self.hidden, bias = False)
        self.fc4 = nn.Linear(self.hidden, self.hidden, bias = False)
        self.fc5 = nn.Linear(self.hidden, self.outputs, bias = False)
        
    def forward(self, x):
        output = self.fc1(x)
        output = F.relu(output)
        output = self.fc2(output)
        output = F.relu(output)
        output = self.fc3(output)
        output = F.relu(output)
        output = self.fc4(output)
        output = F.relu(output)
        output = self.fc5(output)
        return output

In [ ]:
model = FeedForward(input_size, output_size, hidden_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model.parameters() , lr = learning_rate)

start = time.time()

# Train the model
for epoch in range(num_epochs):

    running_loss = 0
    running_error = 0
    num_batches = 0
    
    shuffled_indices = torch.randperm(X_train.size()[0])
 
    for count in range(0, X_train.size()[0], batch_size):

        # forward and backward pass
        optimizer.zero_grad()
        
        indices=shuffled_indices[count:count+batch_size]
        minibatch_data =  X_train[indices]
        minibatch_label = Y_train[indices]

        inputs = minibatch_data.view(minibatch_data.size(),48)
        inputs.requires_grad_()
        outputs = model(inputs) 

        loss = criterion(outputs, minibatch_label)
        loss.backward()
        optimizer.step()
        
        # compute some stats
        running_loss += loss.detach().item()
        error = get_error(outputs.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches += 1
    
    # once the epoch is finished we divide the "running quantities" by the number of batches
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed_time = time.time() - start
    
    # every 10 epoch we display the stats and compute the error rate on the test set  
    if epoch % 10 == 0 : 

        print("For Num. of Epochs: {:.0f}, Time: {:.5f}, Loss: {:.5f}, Learning Rate: {:.5f}, Error: {:.5f} %".format(epoch, elapsed_time, total_loss, learning_rate, total_error * 100))
        eval_on_test_set()

For Num. of Epochs: 0, Time: 0.06032, Loss: 1.09798, Learning Rate: 0.01000, Error: 63.83872 %
Test error: 63.08333 %
For Num. of Epochs: 10, Time: 0.68933, Loss: 1.09094, Learning Rate: 0.01000, Error: 63.82386 %
Test error: 63.08333 %
For Num. of Epochs: 20, Time: 1.33754, Loss: 1.05937, Learning Rate: 0.01000, Error: 44.13972 %
Test error: 41.54167 %
For Num. of Epochs: 30, Time: 1.97048, Loss: 0.49786, Learning Rate: 0.01000, Error: 14.00595 %
Test error: 12.44445 %
For Num. of Epochs: 40, Time: 2.59659, Loss: 0.21819, Learning Rate: 0.01000, Error: 8.99294 %
Test error: 8.63889 %
For Num. of Epochs: 50, Time: 3.25169, Loss: 0.16150, Learning Rate: 0.01000, Error: 6.52917 %
Test error: 13.33333 %
For Num. of Epochs: 60, Time: 3.87615, Loss: 0.10754, Learning Rate: 0.01000, Error: 3.83129 %
Test error: 5.88889 %
For Num. of Epochs: 70, Time: 4.49859, Loss: 0.07830, Learning Rate: 0.01000, Error: 2.68673 %
Test error: 18.66667 %
For Num. of Epochs: 80, Time: 5.13337, Loss: 0.04336, L

In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 6.34779,     mini batch size: 30,     learning rate: 0.0100,     num of hidden nodes: 100
Test error: 0.00000 %


## Results: 
### Constant hyper parameters used: 

  1.   Number of epochs = 100
  2.   Learning rates = 0.001 (Fixed)
  3.   Mini-batch size = 30
  4.   Number of hidden nodes = 100

### Classification Error on Test set for Number of Layers: 

For learning rate = 0.001
- num of layers: 1, Time: 2.83738, Test error: 19.63889 %
- num of layers: 2, Time: 4.19657, Test error: 16.05556 %
- num of layers: 3, Time: 4.65410, Test error: 17.38889 %
- num of layers: 4, Time: 5.61082, Test error: 43.34722 %
- num of layers: 5, Time: 6.28459, Test error: 56.63889 %

For learning rate = 0.01
- num of layers: 1, Time: 2.83572, Test error: 7.09722 %
- num of layers: 2, Time: 4.24768, Test error: 2.55556 %
- num of layers: 3, Time: 4.61948, Test error: 0.66667 %
- num of layers: 4, Time: 5.48521, Test error: 0.33333 %
- num of layers: 5, Time: 6.34779, Test error: 0.00000 %
*NOTE: signs of overfitting in layers 4 and 5. 



## Shallow, Wide and Deep Neural Networks

### Shallow Neural Network
1 hidden layer, 100 hidden nodes

In [ ]:
# Hyper-parameters 
input_size = X_train.shape[1]   # 48
output_size = len(Y_train.unique())   # 3 outcomes: 0, 1, 2
hidden_size = 100  # no of nodes in hidden layers
num_epochs = 100
batch_size = 30
learning_rate = 0.001

# Make a fully connected neural network with 3 layers
class FeedForward(nn.Module):

    def __init__(self, input_size, output_size, hidden_size):
        super(FeedForward, self).__init__()
        self.inputs = input_size
        self.outputs = output_size
        self.hidden = hidden_size

        # Define the layers
        self.fc1 = nn.Linear(self.inputs, self.hidden, bias = False)
        self.fc2 = nn.Linear(self.hidden, self.hidden, bias = False)
        
    def forward(self, x):
        output = self.fc1(x)
        output = F.relu(output)
        output = self.fc2(output)
        return output

In [ ]:
model = FeedForward(input_size, output_size, hidden_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model.parameters() , lr = learning_rate)

start = time.time()

# Train the model
for epoch in range(num_epochs):

    running_loss = 0
    running_error = 0
    num_batches = 0
    
    shuffled_indices = torch.randperm(X_train.size()[0])
 
    for count in range(0, X_train.size()[0], batch_size):

        # forward and backward pass
        optimizer.zero_grad()
        
        indices=shuffled_indices[count:count+batch_size]
        minibatch_data =  X_train[indices]
        minibatch_label = Y_train[indices]

        inputs = minibatch_data.view(minibatch_data.size(),48)
        inputs.requires_grad_()
        outputs = model(inputs) 

        loss = criterion(outputs, minibatch_label)
        loss.backward()
        optimizer.step()
        
        # compute some stats
        running_loss += loss.detach().item()
        error = get_error(outputs.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches += 1
    
    # once the epoch is finished we divide the "running quantities" by the number of batches
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed_time = time.time() - start
    
    # every 10 epoch we display the stats and compute the error rate on the test set  
    if epoch % 10 == 0 : 

        print("For Num. of Epochs: {:.0f}, Time: {:.5f}, Loss: {:.5f}, Learning Rate: {:.5f}, Error: {:.5f} %".format(epoch, elapsed_time, total_loss, learning_rate, total_error * 100))
        eval_on_test_set()

For Num. of Epochs: 0, Time: 0.04393, Loss: 4.57576, Learning Rate: 0.00100, Error: 100.00000 %
Test error: 100.00000 %
For Num. of Epochs: 10, Time: 0.50470, Loss: 2.13814, Learning Rate: 0.00100, Error: 61.81717 %
Test error: 61.08333 %
For Num. of Epochs: 20, Time: 0.91285, Loss: 1.15120, Learning Rate: 0.00100, Error: 48.33147 %
Test error: 51.08333 %
For Num. of Epochs: 30, Time: 1.33616, Loss: 1.06979, Learning Rate: 0.00100, Error: 40.89558 %
Test error: 39.77778 %
For Num. of Epochs: 40, Time: 1.75101, Loss: 1.02867, Learning Rate: 0.00100, Error: 30.30844 %
Test error: 31.22222 %
For Num. of Epochs: 50, Time: 2.18022, Loss: 0.99414, Learning Rate: 0.00100, Error: 26.75214 %
Test error: 22.95833 %
For Num. of Epochs: 60, Time: 2.60200, Loss: 0.95979, Learning Rate: 0.00100, Error: 22.71646 %
Test error: 20.61111 %
For Num. of Epochs: 70, Time: 3.03094, Loss: 0.92372, Learning Rate: 0.00100, Error: 22.98402 %
Test error: 21.38889 %
For Num. of Epochs: 80, Time: 3.45305, Loss: 0.

In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 4.26615,     mini batch size: 30,     learning rate: 0.0010,     num of hidden nodes: 100
Test error: 15.51389 %


### Wide Neural Network
1 Hidden Layer, 400 Hidden Nodes

In [ ]:
# Hyper-parameters 
input_size = X_train.shape[1]   # 48
output_size = len(Y_train.unique())   # 3 outcomes: 0, 1, 2
hidden_size = 400  # no of nodes in hidden layers
num_epochs = 100
batch_size = 30
learning_rate = 0.001

# Make a fully connected neural network with 3 layers
class FeedForward(nn.Module):

    def __init__(self, input_size, output_size, hidden_size):
        super(FeedForward, self).__init__()
        self.inputs = input_size
        self.outputs = output_size
        self.hidden = hidden_size

        # Define the layers
        self.fc1 = nn.Linear(self.inputs, self.hidden, bias = False)
        self.fc2 = nn.Linear(self.hidden, self.hidden, bias = False)
        self.fc3 = nn.Linear(self.hidden, self.outputs, bias = False)
        
    def forward(self, x):
        output = self.fc1(x)
        output = F.relu(output)
        output = self.fc2(output)
        output = F.relu(output)
        output = self.fc3(output)
        return output

In [ ]:
model = FeedForward(input_size, output_size, hidden_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model.parameters() , lr = learning_rate)

start = time.time()

# Train the model
for epoch in range(num_epochs):

    running_loss = 0
    running_error = 0
    num_batches = 0
    
    shuffled_indices = torch.randperm(X_train.size()[0])
 
    for count in range(0, X_train.size()[0], batch_size):

        # forward and backward pass
        optimizer.zero_grad()
        
        indices=shuffled_indices[count:count+batch_size]
        minibatch_data =  X_train[indices]
        minibatch_label = Y_train[indices]

        inputs = minibatch_data.view(minibatch_data.size(),48)
        inputs.requires_grad_()
        outputs = model(inputs) 

        loss = criterion(outputs, minibatch_label)
        loss.backward()
        optimizer.step()
        
        # compute some stats
        running_loss += loss.detach().item()
        error = get_error(outputs.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches += 1
    
    # once the epoch is finished we divide the "running quantities" by the number of batches
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed_time = time.time() - start
    
    # every 10 epoch we display the stats and compute the error rate on the test set  
    if epoch % 10 == 0 : 

        print("For Num. of Epochs: {:.0f}, Time: {:.5f}, Loss: {:.5f}, Learning Rate: {:.5f}, Error: {:.5f} %".format(epoch, elapsed_time, total_loss, learning_rate, total_error * 100))
        eval_on_test_set()

For Num. of Epochs: 0, Time: 0.11435, Loss: 1.10392, Learning Rate: 0.00100, Error: 70.49424 %
Test error: 70.41667 %
For Num. of Epochs: 10, Time: 1.24530, Loss: 1.07794, Learning Rate: 0.00100, Error: 49.94797 %
Test error: 49.00000 %
For Num. of Epochs: 20, Time: 2.36674, Loss: 1.05880, Learning Rate: 0.00100, Error: 41.11483 %
Test error: 39.69444 %
For Num. of Epochs: 30, Time: 3.45325, Loss: 1.03730, Learning Rate: 0.00100, Error: 35.52583 %
Test error: 37.22222 %
For Num. of Epochs: 40, Time: 4.53970, Loss: 1.00971, Learning Rate: 0.00100, Error: 30.81011 %
Test error: 31.45833 %
For Num. of Epochs: 50, Time: 5.65794, Loss: 0.97369, Learning Rate: 0.00100, Error: 24.20661 %
Test error: 25.15278 %
For Num. of Epochs: 60, Time: 6.78702, Loss: 0.92615, Learning Rate: 0.00100, Error: 18.08993 %
Test error: 16.50000 %
For Num. of Epochs: 70, Time: 7.90686, Loss: 0.86582, Learning Rate: 0.00100, Error: 13.32219 %
Test error: 12.62500 %
For Num. of Epochs: 80, Time: 9.00503, Loss: 0.79

In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 11.11809,     mini batch size: 30,     learning rate: 0.0010,     num of hidden nodes: 400
Test error: 8.98611 %


### Deep Neural Network
3 Hidden Layers and 100 Hidden Nodes

In [ ]:
# Hyper-parameters 
input_size = X_train.shape[1]   # 48
output_size = len(Y_train.unique())   # 3 outcomes: 0, 1, 2
hidden_size = 100  # no of nodes in hidden layers
num_epochs = 1000
batch_size = 30
learning_rate = 0.001

# Fully connected neural network with 5 layers
class FeedForward(nn.Module):

    def __init__(self, input_size, output_size, hidden_size):
        super(FeedForward, self).__init__()
        self.inputs = input_size
        self.outputs = output_size
        self.hidden = hidden_size

        # Define the layers
        self.fc1 = nn.Linear(self.inputs, self.hidden, bias = False)
        self.fc2 = nn.Linear(self.hidden, self.hidden, bias = False)
        self.fc3 = nn.Linear(self.hidden, self.hidden, bias = False)
        self.fc4 = nn.Linear(self.hidden, self.hidden, bias = False)
        self.fc5 = nn.Linear(self.hidden, self.outputs, bias = False)

        
    def forward(self, x):
        output = self.fc1(x)
        output = F.relu(output)
        output = self.fc2(output)
        output = F.relu(output)
        output = self.fc3(output)
        output = F.relu(output)
        output = self.fc4(output)
        output = F.relu(output)
        output = self.fc5(output)
        return output

In [ ]:
model = FeedForward(input_size, output_size, hidden_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model.parameters() , lr = learning_rate)

start = time.time()

# Train the model
for epoch in range(num_epochs):

    running_loss = 0
    running_error = 0
    num_batches = 0
    
    shuffled_indices = torch.randperm(X_train.size()[0])
 
    for count in range(0, X_train.size()[0], batch_size):

        # forward and backward pass
        optimizer.zero_grad()
        
        indices=shuffled_indices[count:count+batch_size]
        minibatch_data =  X_train[indices]
        minibatch_label = Y_train[indices]

        inputs = minibatch_data.view(minibatch_data.size(),48)
        inputs.requires_grad_()
        outputs = model(inputs) 

        loss = criterion(outputs, minibatch_label)
        loss.backward()
        optimizer.step()
        
        # compute some stats
        running_loss += loss.detach().item()
        error = get_error(outputs.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches += 1
    
    # once the epoch is finished we divide the "running quantities" by the number of batches
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed_time = time.time() - start
    
    # every 100 epoch we display the stats and compute the error rate on the test set  
    if epoch % 100 == 0 : 

        print("For Num. of Epochs: {:.0f}, Time: {:.5f}, Loss: {:.5f}, Learning Rate: {:.5f}, Error: {:.5f} %".format(epoch, elapsed_time, total_loss, learning_rate, total_error * 100))
        eval_on_test_set()

For Num. of Epochs: 0, Time: 0.06783, Loss: 1.09900, Learning Rate: 0.00100, Error: 66.72612 %
Test error: 66.20833 %
For Num. of Epochs: 100, Time: 6.08217, Loss: 1.09450, Learning Rate: 0.00100, Error: 60.90673 %
Test error: 60.86111 %
For Num. of Epochs: 200, Time: 12.08068, Loss: 1.08387, Learning Rate: 0.00100, Error: 65.71163 %
Test error: 66.38889 %
For Num. of Epochs: 300, Time: 18.09626, Loss: 1.00164, Learning Rate: 0.00100, Error: 65.67447 %
Test error: 66.38889 %
For Num. of Epochs: 400, Time: 24.06647, Loss: 0.31375, Learning Rate: 0.00100, Error: 10.65032 %
Test error: 10.65278 %
For Num. of Epochs: 500, Time: 30.23740, Loss: 0.17204, Learning Rate: 0.00100, Error: 7.26124 %
Test error: 7.20833 %
For Num. of Epochs: 600, Time: 36.33602, Loss: 0.10802, Learning Rate: 0.00100, Error: 4.49275 %
Test error: 4.33333 %
For Num. of Epochs: 700, Time: 42.28646, Loss: 0.05968, Learning Rate: 0.00100, Error: 1.31178 %
Test error: 1.22222 %
For Num. of Epochs: 800, Time: 48.20439, L

In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 1000,     Time: 60.06633,     mini batch size: 30,     learning rate: 0.0010,     num of hidden nodes: 100
Test error: 0.00000 %


## Summary:

### Constant hyper parameters used: 
  1.   Number of epochs = 100
  2.   Learning rates = 0.001 (Fixed)
  3.   Mini-batch size = 30

### Shallow Neural Network:
- 2 hidden layers
- 100 hidden nodes
- Time: 4.26615
- Test error: 15.51389 %

### Wide Neural Network:
- 2 hidden layers
- 400 hidden nodes
- Time: 11.11809
- Test error: 8.98611 %

### Deep Neural Network:
- 4 hidden layers
- 200 hidden nodes
- Time: 6.18328
- Test error: 59.20833 %

# Task 3: 
You are asked to study the effect of learning rates. As with Task 2, your experiments have to be well documented. You need to give correct conclusion and give suggestion how learning rates should be set. This includes possible adaptive learning rates where the value increases or decreases as the increase of epochs

## Changing the learning rates

In [ ]:
# Hyper-parameters 
input_size = X_train.shape[1]   # 48
output_size = len(Y_train.unique())   # 3 outcomes: 0, 1, 2
hidden_size = 100  # no of nodes in hidden layers
num_epochs = 100
batch_size = 30
learning_rate = 0.001

# Make a fully connected neural network with 3 layers
class FeedForward(nn.Module):

    def __init__(self, input_size, output_size, hidden_size):
        super(FeedForward, self).__init__()
        self.inputs = input_size
        self.outputs = output_size
        self.hidden = hidden_size

        # Define the layers
        self.fc1 = nn.Linear(self.inputs, self.hidden, bias = False)
        self.fc2 = nn.Linear(self.hidden, self.hidden, bias = False)
        self.fc3 = nn.Linear(self.hidden, self.outputs, bias = False)
        
    def forward(self, x):
        output = self.fc1(x)
        output = F.relu(output)
        output = self.fc2(output)
        output = F.relu(output)
        output = self.fc3(output)
        return output

In [ ]:
model = FeedForward(input_size, output_size, hidden_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model.parameters() , lr = learning_rate)

start = time.time()

# Train the model
for epoch in range(num_epochs):

    running_loss = 0
    running_error = 0
    num_batches = 0
    
    shuffled_indices = torch.randperm(X_train.size()[0])
 
    for count in range(0, X_train.size()[0], batch_size):

        # forward and backward pass
        optimizer.zero_grad()
        
        indices=shuffled_indices[count:count+batch_size]
        minibatch_data =  X_train[indices]
        minibatch_label = Y_train[indices]

        inputs = minibatch_data.view(minibatch_data.size(),48)
        inputs.requires_grad_()
        outputs = model(inputs) 

        loss = criterion(outputs, minibatch_label)
        loss.backward()
        optimizer.step()
        
        # compute some stats
        running_loss += loss.detach().item()
        error = get_error(outputs.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches += 1
    
    # once the epoch is finished we divide the "running quantities" by the number of batches
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed_time = time.time() - start
    
    # every 10 epoch we display the stats and compute the error rate on the test set  
    if epoch % 10 == 0 : 

        print("For Num. of Epochs: {:.0f}, Time: {:.5f}, Loss: {:.5f}, Learning Rate: {:.5f}, Error: {:.5f} %".format(epoch, elapsed_time, total_loss, learning_rate, total_error * 100))
        eval_on_test_set()

For Num. of Epochs: 0, Time: 0.04272, Loss: 1.09841, Learning Rate: 0.00100, Error: 70.59457 %
Test error: 70.75000 %
For Num. of Epochs: 10, Time: 0.50386, Loss: 1.08973, Learning Rate: 0.00100, Error: 58.61390 %
Test error: 59.11111 %
For Num. of Epochs: 20, Time: 0.94285, Loss: 1.08186, Learning Rate: 0.00100, Error: 50.96618 %
Test error: 51.58333 %
For Num. of Epochs: 30, Time: 1.40290, Loss: 1.07357, Learning Rate: 0.00100, Error: 43.86473 %
Test error: 44.47222 %
For Num. of Epochs: 40, Time: 1.88326, Loss: 1.06418, Learning Rate: 0.00100, Error: 44.88666 %
Test error: 46.44444 %
For Num. of Epochs: 50, Time: 2.36523, Loss: 1.05287, Learning Rate: 0.00100, Error: 45.31401 %
Test error: 46.00000 %
For Num. of Epochs: 60, Time: 2.81724, Loss: 1.03862, Learning Rate: 0.00100, Error: 43.21442 %
Test error: 43.44444 %
For Num. of Epochs: 70, Time: 3.28719, Loss: 1.01985, Learning Rate: 0.00100, Error: 40.68004 %
Test error: 41.33333 %
For Num. of Epochs: 80, Time: 3.74420, Loss: 0.99

In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 4.62454,     mini batch size: 30,     learning rate: 0.0010,     num of hidden nodes: 100
Test error: 17.61111 %


In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 4.64185,     mini batch size: 30,     learning rate: 0.0030,     num of hidden nodes: 100
Test error: 7.09722 %


In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 4.72465,     mini batch size: 30,     learning rate: 0.0050,     num of hidden nodes: 100
Test error: 6.20833 %


In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 4.84306,     mini batch size: 30,     learning rate: 0.0070,     num of hidden nodes: 100
Test error: 2.22222 %


In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 4.69736,     mini batch size: 30,     learning rate: 0.0090,     num of hidden nodes: 100
Test error: 0.77778 %


## Results: 
### Constant hyper parameters used: 

  1.   Number of epochs = 100
  2.   Mini-batch size = 30
  3.   Number of hidden nodes = 100


### Classification Error on Test set for Fixed Learning Rates: 
- learning rate: 0.0010, Time: 4.62454, Test error: 17.61111 %
- learning rate: 0.0030, Time: 4.64185, Test error: 7.09722 %
- learning rate: 0.0050, Time: 4.72465, Test error: 6.20833 %
- learning rate: 0.0070, Time: 4.84306, Test error: 2.22222 %
- learning rate: 0.0090, Time: 4.69736, Test error: 0.77778 %






## Adaptive learning rates
Divide the learning rate by `x` every 10 epochs
- when x > 1, learning rate decreases with the number of epochs
- when x < 1, learning rate increases with the number of epochs

In [ ]:
# Hyper-parameters 
input_size = X_train.shape[1]   # 48
output_size = len(Y_train.unique())   # 3 outcomes: 0, 1, 2
hidden_size = 100  # no of nodes in hidden layers
num_epochs = 100
batch_size = 30
learning_rate = 0.005

# Make a fully connected neural network with 3 layers
class FeedForward(nn.Module):

    def __init__(self, input_size, output_size, hidden_size):
        super(FeedForward, self).__init__()
        self.inputs = input_size
        self.outputs = output_size
        self.hidden = hidden_size

        # Define the layers
        self.fc1 = nn.Linear(self.inputs, self.hidden, bias = False)
        self.fc2 = nn.Linear(self.hidden, self.hidden, bias = False)
        self.fc3 = nn.Linear(self.hidden, self.outputs, bias = False)
        
    def forward(self, x):
        output = self.fc1(x)
        output = F.relu(output)
        output = self.fc2(output)
        output = F.relu(output)
        output = self.fc3(output)
        return output

In [ ]:
model = FeedForward(input_size, output_size, hidden_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model.parameters() , lr = learning_rate)

start = time.time()

# Train the model
for epoch in range(num_epochs):

    # learning rate strategy : divide the learning rate by x every 10 epochs
    x = 1.5   # Change this
    if epoch % 10 == 0 and epoch > 10: 
        learning_rate = learning_rate / x
    
    # create a new optimizer at the beginning of each epoch: give the current learning rate.   
    optimizer = torch.optim.SGD(params = model.parameters() , lr = learning_rate)
        
    running_loss = 0
    running_error = 0
    num_batches = 0
    
    shuffled_indices = torch.randperm(X_train.size()[0])
 
    for count in range(0, X_train.size()[0], batch_size):
        
        # forward and backward pass
        optimizer.zero_grad()
        
        indices=shuffled_indices[count:count+batch_size]
        minibatch_data =  X_train[indices]
        minibatch_label = Y_train[indices]

        inputs = minibatch_data.view(minibatch_data.size(),48)
        inputs.requires_grad_()
        outputs = model(inputs) 

        loss = criterion(outputs, minibatch_label)
        loss.backward()
        optimizer.step()
        
        # compute some stats
        running_loss += loss.detach().item()
        error = get_error(outputs.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches += 1
    
    
    # once the epoch is finished we divide the "running quantities"
    # by the number of batches
    
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed_time = time.time() - start
    
    # every 10 epoch we display the stats 
    # and compute the error rate on the test set  
    
    if epoch % 10 == 0 : 
            
        print("For Num. of Epochs: {:.0f}, Time: {:.5f}, Loss: {:.5f}, Learning Rate: {:.5f}, Error: {:.5f} %".format(epoch, elapsed_time, total_loss, learning_rate, total_error * 100))
        
        eval_on_test_set()

        

For Num. of Epochs: 0, Time: 0.04356, Loss: 1.09902, Learning Rate: 0.00500, Error: 66.05351 %
Test error: 63.09722 %
For Num. of Epochs: 10, Time: 0.50549, Loss: 1.07140, Learning Rate: 0.00500, Error: 50.61315 %
Test error: 49.68055 %
For Num. of Epochs: 20, Time: 0.95956, Loss: 1.01333, Learning Rate: 0.00333, Error: 43.04348 %
Test error: 43.34722 %
For Num. of Epochs: 30, Time: 1.42415, Loss: 0.90432, Learning Rate: 0.00222, Error: 25.49981 %
Test error: 23.13889 %
For Num. of Epochs: 40, Time: 1.88610, Loss: 0.77574, Learning Rate: 0.00148, Error: 16.57748 %
Test error: 14.16667 %
For Num. of Epochs: 50, Time: 2.35708, Loss: 0.67903, Learning Rate: 0.00099, Error: 13.88703 %
Test error: 12.62500 %
For Num. of Epochs: 60, Time: 2.81721, Loss: 0.61837, Learning Rate: 0.00066, Error: 13.41137 %
Test error: 12.06944 %
For Num. of Epochs: 70, Time: 3.26433, Loss: 0.58085, Learning Rate: 0.00044, Error: 12.67187 %
Test error: 11.86111 %
For Num. of Epochs: 80, Time: 3.73191, Loss: 0.55

Starting Learning Rate: 0.01

In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    x: {:.1f}, \
    ending learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, x, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 4.61761,     mini batch size: 30,     x: 1.3,     ending learning rate: 0.0012,     num of hidden nodes: 100
Test error: 4.22222 %


In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    x: {:.1f}, \
    ending learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, x, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 4.66165,     mini batch size: 30,     x: 1.4,     ending learning rate: 0.0007,     num of hidden nodes: 100
Test error: 6.22222 %


In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    x: {:.1f}, \
    ending learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, x, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 4.65058,     mini batch size: 30,     x: 1.5,     ending learning rate: 0.0004,     num of hidden nodes: 100
Test error: 7.20833 %


Starting Learning Rate: 0.007

In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    x: {:.1f}, \
    ending learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, x, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 4.52604,     mini batch size: 30,     x: 1.3,     ending learning rate: 0.0009,     num of hidden nodes: 100
Test error: 7.76389 %


In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    x: {:.1f}, \
    ending learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, x, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 4.63525,     mini batch size: 30,     x: 1.4,     ending learning rate: 0.0005,     num of hidden nodes: 100
Test error: 8.54167 %


In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    x: {:.1f}, \
    ending learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, x, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 4.79620,     mini batch size: 30,     x: 1.5,     ending learning rate: 0.0003,     num of hidden nodes: 100
Test error: 10.08333 %


Starting Learning Rate: 0.005

In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    x: {:.1f}, \
    ending learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, x, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 4.64237,     mini batch size: 30,     x: 0.8,     ending learning rate: 0.0298,     num of hidden nodes: 100
Test error: 1.22222 %


In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    x: {:.1f}, \
    ending learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, x, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 4.65208,     mini batch size: 30,     x: 0.9,     ending learning rate: 0.0116,     num of hidden nodes: 100
Test error: 1.77778 %


In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    x: {:.1f}, \
    ending learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, x, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 4.56385,     mini batch size: 30,     x: 1.3,     ending learning rate: 0.0006,     num of hidden nodes: 100
Test error: 8.20833 %


In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    x: {:.1f}, \
    ending learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, x, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 4.64512,     mini batch size: 30,     x: 1.4,     ending learning rate: 0.0003,     num of hidden nodes: 100
Test error: 10.09722 %


In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    x: {:.1f}, \
    ending learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, x, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 4.61243,     mini batch size: 30,     x: 1.5,     ending learning rate: 0.0002,     num of hidden nodes: 100
Test error: 12.06945 %


Starting Learning Rate: 0.003

In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    x: {:.1f}, \
    ending learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, x, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 4.60890,     mini batch size: 30,     x: 0.8,     ending learning rate: 0.0179,     num of hidden nodes: 100
Test error: 3.44445 %


In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    x: {:.1f}, \
    ending learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, x, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 4.66232,     mini batch size: 30,     x: 0.9,     ending learning rate: 0.0070,     num of hidden nodes: 100
Test error: 5.00000 %


## Results: 
### Constant hyper parameters used: 

  1.   Number of epochs = 100
  2.   Mini-batch size = 30
  3.   Number of hidden nodes = 100

Divide the learning rate by `x` every 10 epochs
### Classification Error on Test set for adaptive learning rates:
For Starting Learning Rate = 0.01
- x: 1.3, Time: 4.61761, Test error: 4.22222 %
- x: 1.4, Time: 4.66165, Test error: 6.22222 %
- x: 1.5, Time: 4.65058, Test error: 7.20833 %

For Starting Learning Rate = 0.007
- x: 1.3, Time: 4.52604, Test error: 7.76389 %
- x: 1.4, Time: 4.63525, Test error: 8.54167 %
- x: 1.5, Time: 4.79620, Test error: 10.08333 %

For Starting Learning Rate = 0.005
- x: 0.8, Time: 4.64237, Test error: 1.22222 %
- x: 0.9, Time: 4.65208, Test error: 1.77778 %
- x: 1.3, Time: 4.56385, Test error: 8.20833 %
- x: 1.4, Time: 4.64512, Test error: 10.09722 %
- x: 1.5, Time: 4.61243, Test error: 12.06945 %

For Starting Learning Rate = 0.003
- x: 0.8, Time: 4.60890, Test error: 3.44445 %
- x: 0.9, Time: 4.66232, Test error: 5.00000 %

# Task 4: 
You are asked to study the effect of mini-batch size. You can set mini-batch size to be 1 (stochastic
gradient descent), N (batch gradient descent) or any other size. The most important aspect is to be
conclusive with your finding. The mini-batch size really depends on the problem size. 

### Changing mini-batch size

In [ ]:
# Hyper-parameters 
input_size = X_train.shape[1]   # 48
output_size = len(Y_train.unique())   # 3 outcomes: 0, 1, 2
hidden_size = 100  # no of nodes in hidden layers
num_epochs = 100
batch_size = 50
learning_rate = 0.001

# Make a fully connected neural network with 3 layers
class FeedForward(nn.Module):

    def __init__(self, input_size, output_size, hidden_size):
        super(FeedForward, self).__init__()
        self.inputs = input_size
        self.outputs = output_size
        self.hidden = hidden_size

        # Define the layers
        self.fc1 = nn.Linear(self.inputs, self.hidden, bias = False)
        self.fc2 = nn.Linear(self.hidden, self.hidden, bias = False)
        self.fc3 = nn.Linear(self.hidden, self.outputs, bias = False)
        
    def forward(self, x):
        output = self.fc1(x)
        output = F.relu(output)
        output = self.fc2(output)
        output = F.relu(output)
        output = self.fc3(output)
        return output

In [ ]:
model = FeedForward(input_size, output_size, hidden_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model.parameters() , lr = learning_rate)

start = time.time()

# Train the model
for epoch in range(num_epochs):

    running_loss = 0
    running_error = 0
    num_batches = 0
    
    shuffled_indices = torch.randperm(X_train.size()[0])
 
    for count in range(0, X_train.size()[0], batch_size):

        # forward and backward pass
        optimizer.zero_grad()
        
        indices=shuffled_indices[count:count+batch_size]
        minibatch_data =  X_train[indices]
        minibatch_label = Y_train[indices]

        inputs = minibatch_data.view(minibatch_data.size(),48)
        inputs.requires_grad_()
        outputs = model(inputs) 

        loss = criterion(outputs, minibatch_label)
        loss.backward()
        optimizer.step()
        
        # compute some stats
        running_loss += loss.detach().item()
        error = get_error(outputs.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches += 1
    
    # once the epoch is finished we divide the "running quantities" by the number of batches
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed_time = time.time() - start
    
    # every 10 epoch we display the stats and compute the error rate on the test set  
    if epoch % 10 == 0 : 

        print("For Num. of Epochs: {:.0f}, Time: {:.5f}, Loss: {:.5f}, Learning Rate: {:.5f}, Error: {:.5f} %".format(epoch, elapsed_time, total_loss, learning_rate, total_error * 100))
        eval_on_test_set()

For Num. of Epochs: 0, Time: 0.03053, Loss: 1.09714, Learning Rate: 0.00100, Error: 65.38690 %
Test error: 64.66049 %
For Num. of Epochs: 10, Time: 0.32883, Loss: 1.09269, Learning Rate: 0.00100, Error: 61.10119 %
Test error: 59.61728 %
For Num. of Epochs: 20, Time: 0.62057, Loss: 1.08697, Learning Rate: 0.00100, Error: 52.92857 %
Test error: 51.88889 %
For Num. of Epochs: 30, Time: 0.92437, Loss: 1.08270, Learning Rate: 0.00100, Error: 46.25000 %
Test error: 45.17901 %
For Num. of Epochs: 40, Time: 1.22828, Loss: 1.07742, Learning Rate: 0.00100, Error: 41.95833 %
Test error: 40.17901 %
For Num. of Epochs: 50, Time: 1.52014, Loss: 1.07216, Learning Rate: 0.00100, Error: 40.73214 %
Test error: 39.29012 %
For Num. of Epochs: 60, Time: 1.83593, Loss: 1.06532, Learning Rate: 0.00100, Error: 39.03571 %
Test error: 38.80247 %
For Num. of Epochs: 70, Time: 2.13391, Loss: 1.05901, Learning Rate: 0.00100, Error: 38.66667 %
Test error: 37.60494 %
For Num. of Epochs: 80, Time: 2.42861, Loss: 1.05

In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 12.06189,     mini batch size: 10,     learning rate: 0.0010,     num of hidden nodes: 100
Test error: 7.71536 %


In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 6.64465,     mini batch size: 20,     learning rate: 0.0010,     num of hidden nodes: 100
Test error: 13.14815 %


In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 4.61398,     mini batch size: 30,     learning rate: 0.0010,     num of hidden nodes: 100
Test error: 29.48611 %


In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 3.64458,     mini batch size: 40,     learning rate: 0.0010,     num of hidden nodes: 100
Test error: 38.04348 %


In [ ]:
print("For num of epochs: {:.0f}, \
    Time: {:.5f}, \
    mini batch size: {:.0f}, \
    learning rate: {:.4f}, \
    num of hidden nodes: {:.0f}".format(num_epochs, elapsed_time, batch_size, learning_rate, hidden_size))
eval_on_test_set()

For num of epochs: 100,     Time: 3.00074,     mini batch size: 50,     learning rate: 0.0010,     num of hidden nodes: 100
Test error: 38.42593 %


### Results: 
### Constant hyper parameters used: 

  1.   Number of Epochs = 100
  2.   Learning rates = 0.001 (Fixed)
  3.   Number of hidden nodes = 100


### Classification Error on Test set for mini batch size: 
- mini batch size: 10, Time: 12.06189, Test error: 7.71536 %
- mini batch size: 20, Time: 6.64465, Test error: 13.14815 %
- mini batch size: 30, Time: 4.61398, Test error: 29.48611 %
- mini batch size: 40, Time: 3.64458, Test error: 38.04348 %
- mini batch size: 50, Time: 3.00074, Test error: 38.42593 %


